Copied from `mkid/multichroic/wafers/MKIDArray02/0001`

In [1]:
from __future__ import division, print_function
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

import os
from glob import glob
from kid_readout.measurement.io import npy
from kid_readout.analysis.resonator import lmfit_resonator

In [2]:
roots = {
    '200': '/data/readout/2017-12-04_111009_mmw.npd/',
    '220': '/data/readout/2017-12-04_113210_mmw.npd/',
    '240': '/data/readout/2017-12-04_114944_mmw.npd/',
    '160': '/data/readout/2017-12-05_123750_mmw.npd/',
    '180': '/data/readout/2017-12-05_125417_mmw.npd/',
    '190': '/data/readout/2017-12-05_130953_mmw.npd/',
    '260': '/data/readout/2017-12-05_132400_mmw.npd/',
    '140': '/data/readout/2017-12-06_115910_mmw.npd/',
    }

In [10]:
for root in roots.values():
    npd = npy.NumpyDirectory(root)
    filename = os.path.split(npd.root_path)[-1] + '.pkl'
    if glob(filename):
        print("Exists: {}".format(filename))
    else:
        ssl_list = [npd.read(name) for name in npd.node_names() if name.startswith('SweepStreamList')]
        ssl_dfs = []
        for ssl in ssl_list:  # One SweepStreamList per band
            if (root == roots['200'] and
                ssl.state.band_index in (6, 7) and
                ssl.state.attenuation_index == 0):
                continue
            band_dfs = []
            num_resonators = ssl.sweep.num_channels - ssl.state.num_dummy_frequencies
            for number in range(num_resonators):
                sssl = ssl[number]
                sssl.single_sweep.fit_resonator(model=lmfit_resonator.LinearLossResonatorWithCable)
                sssl.single_sweep.state.update(sssl.state)
                band_dfs.append(sssl.single_sweep.to_dataframe())
            ssl_dfs.append(pd.concat(band_dfs, ignore_index=True))
        df = pd.concat(ssl_dfs, ignore_index=True)
        df.to_pickle(filename)
        print("Saved: {}".format(filename))
    npd.close()

Exists: 2017-12-04_111009_mmw.npd.pkl
Saved: 2017-12-05_125417_mmw.npd.pkl
Saved: 2017-12-05_132400_mmw.npd.pkl
Saved: 2017-12-04_114944_mmw.npd.pkl
Saved: 2017-12-06_115910_mmw.npd.pkl
Saved: 2017-12-05_123750_mmw.npd.pkl
Saved: 2017-12-05_130953_mmw.npd.pkl
Saved: 2017-12-04_113210_mmw.npd.pkl
